# Data preprocessing

In [1]:
# Import libraries
import pandas as pd
import os
import csv
import billboard
import lyricsgenius
import json
import re

In [2]:
# Loading data and adding year columns and wrapping up billboard songs (1950-2015)
cwd = os.getcwd()
data_path = os.chdir(cwd + "\data")
all_csv_files = os.listdir(data_path)
songs_50_15 = []

for filename in all_csv_files:
    songs = pd.read_csv(filename, header = 0, sep = ';')#, error_bad_lines=False)
    year = os.path.basename(filename)[:-4]
    songs['Year'] = year
    songs_50_15.append(songs)

In [3]:
# Downloading data from years 2016-2020 from billboard library
playlist = 'hot-100-songs'
dates = [2016, 2017, 2018, 2019, 2020]
songs_billboard = []
for y in dates:
    songs = billboard.ChartData(playlist, date = None, year = y, fetch = True, timeout = 25)
    songs_billboard.append(songs)

In [4]:
# Preprocessing data from years 2016-2020
song_titles = []
artists = []
for year in songs_billboard:
    for playlist in year:
        song = str(playlist).split(" by ")
        song_titles.append(song[0].strip("\'"))
        artists.append(song[1])
        
# Adding missed song on 87th position in 2016 chart
song_titles.insert(86, 'All the Way Up')
artists.insert(86, 'Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infared')

In [5]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Splitting list into same-length elements lists
def list_split(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

split_artists = list(list_split(artists, 100))
split_song_titles = list(list_split(song_titles, 100))


# Creating list of dictionaries for each year to convert them into pandas DataFrame
def dicts_of_dates(lst_of_dates):
    songs = []
    for i in range(len(lst_of_dates)):
        ranger = list(range(1, 101))
        year = lst_of_dates[i]
        dict_ = {'Position': ranger, 'Artist': split_artists[i], 'Song Title': split_song_titles[i], 'Year': [dates[i] for year in range(1, 101)]}
        as_pandas_df = pd.DataFrame(dict_)
        songs.append(as_pandas_df)
    return songs

songs_16_20 = dicts_of_dates(dates)
all_songs = songs_50_15 + songs_16_20
all_time_billboard_wrap_up = pd.concat(all_songs)
all_time_billboard_wrap_up.reset_index(drop = True, inplace = True) # reset indexing
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950
1,2,Nat King Cole,Mona Lisa,1950
2,3,Anton Karas,Third Man Theme,1950
3,4,Gary and Bing Crosby,Sam's Song,1950
4,5,Gary and Bing Crosby,Simple Melody,1950
...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020
6676,97,Luke Combs,Lovin' On You,2020
6677,98,Moneybagg Yo,Said Sum,2020
6678,99,H.E.R. Featuring YG,Slide,2020


In [6]:
# Pandas dataframe nans checking
print(all_time_billboard_wrap_up['Position'].isna().any())
print(all_time_billboard_wrap_up['Artist'].isna().any())
print(all_time_billboard_wrap_up['Song Title'].isna().any())
print(all_time_billboard_wrap_up['Year'].isna().any())

False
False
False
False


# Genius lyrics

In [7]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Spliting authors by 'and' and 'Featuring'
artists = list(all_time_billboard_wrap_up.loc[:,"Artist"])
split_artists = []
for artist in artists:
    if any(re.findall(r'and|Featuring', str(artist))):
        result = re.split(r'and|Featuring', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

all_time_billboard_wrap_up['Split names'] = split_artists
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year,Split names
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,Gordon Jenkins
1,2,Nat King Cole,Mona Lisa,1950,Nat King Cole
2,3,Anton Karas,Third Man Theme,1950,Anton Karas
3,4,Gary and Bing Crosby,Sam's Song,1950,Gary
4,5,Gary and Bing Crosby,Simple Melody,1950,Gary
...,...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020,Morgan Wallen
6676,97,Luke Combs,Lovin' On You,2020,Luke Combs
6677,98,Moneybagg Yo,Said Sum,2020,Moneybagg Yo
6678,99,H.E.R. Featuring YG,Slide,2020,H.E.R.


In [28]:
# lyricsgenius settings
genius_token = os.environ.get('genius_token')
genius = lyricsgenius.Genius(genius_token, timeout = 30, retries = 3)
songs = list(all_time_billboard_wrap_up.loc[:, "Song Title"])
artists = list(all_time_billboard_wrap_up.loc[:,"Split names"])

# Saving lyrics to jsons function
def return_lyrics_json(song, artist, json_name):
    song_lyrics = genius.search_song(song, artist)
    with open(json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': song_lyrics.lyrics}, f, ensure_ascii = False, indent = 4)

# No lyrics function (only instrumental songs)
def no_lyrics(json_name):
    with open(json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': ""}, f, ensure_ascii = False, indent = 4)
  

In [8]:
'''
error_list = []
for i in range(len(songs)):
    try:
        return_lyrics_json(songs[i], artists[i], str(i)+'.json')
    except AttributeError:
        no_lyrics(str(i)+'.json')
        error_list.append(i)
'''

Searching for "Goodnight Irene" by Gordon Jenkins...
Done.
Searching for "Mona Lisa" by Nat King Cole...
Done.
Searching for "Third Man Theme" by Anton Karas...
Done.
Searching for "Sam's Song" by Gary...
Done.
Searching for "Simple Melody" by Gary...
Done.
Searching for "Music, Music, Music" by Teresa Brewer...
Done.
Searching for "Third Man Theme" by Guy Lombardo...
Done.
Searching for "Chattanoogie Shoe Shine Boy" by Red Foley...
Done.
Searching for "Harbor Lights" by Sammy Kaye...
Done.
Searching for "It Isn't Fair" by Sammy Kaye...
Done.
Searching for "If I Knew You Were Coming I'd've Baked A Cake" by Eileen Barton...
Done.
Searching for "Bonaparte's Retreat" by Kay Starr...
Done.
Searching for "Tzena, Tzena, Tzena" by Gordon Jenkins...
No results found for: 'Tzena, Tzena, Tzena Gordon Jenkins'
Searching for "There's No Tomorrow" by Tony Martin...
Done.
Searching for "The Thing" by Phil Harris...
Done.
Searching for "Sentimental Me" by Ames Brothers...
Done.
Searching for "I Wanna

Done.
Searching for "That's Amore" by Dean Martin...
Done.
Searching for "The Happy Wanderer" by Frank Weir...
Done.
Searching for "Answer Me My Love" by Nat King Cole...
Done.
Searching for "Stranger In Paradise" by Tony Bennett...
Done.
Searching for "If I Give My Heart To You" by Doris Day...
Done.
Searching for "If You Love Me (Really Love Me)" by Kay Starr...
Done.
Searching for "Skokiaan" by Ralph Marterie...
No results found for: 'Skokiaan Ralph Marterie'
Searching for "Hold My Hand" by Don Cornell...
Done.
Searching for "Changing Partners" by Patti Page...
Done.
Searching for "Papa Loves Mambo" by Perry Como...
Done.
Searching for "Shake, Rattle And Roll" by Bill Haley...
Done.
Searching for "Rags To Riches" by Tony Bennett...
Done.
Searching for "In The Chapel In The Moonlight" by Kitty Kallen...
Done.
Searching for "Stranger In Paradise" by Four Aces...
Done.
Searching for "Here" by Tony Martin...
Done.
Searching for "Cherry Pink And Apple Blossom White" by Perez Prado...
Don

Done.
Searching for "Moritat" by Richard Hayman...
No results found for: 'Moritat Richard Hayman'
Searching for "Night Lights" by Nat King Cole...
Done.
Searching for "Speedoo" by Cadillacs...
Done.
Searching for "The Church Bells May Ring" by Diamonds...
Done.
Searching for "Mr Wonderful" by Peggy Lee...
Done.
Searching for "My Blue Heaven" by Fats Domino...
Done.
Searching for "Treasure of Love" by Clyde McPhatter...
Done.
Searching for "Miracle of Love" by Eileen Rodgers...
Done.
Searching for "I Want You to Be My Girl" by Frankie Lymon...
Done.
Searching for "Tonight You Belong to Me" by Lawrence Welk...
Done.
Searching for "Lay Down Your Arms" by Chordettes...
Done.
Searching for "Theme From 'the Man with the Golden Arm" by Richard Maltby...
No results found for: 'Theme From 'the Man with the Golden Arm Richard Maltby'
Searching for "Bo Weevil" by Teresa Brewer...
Done.
Searching for "Lullaby of Birdland" by Blue Stars...
Done.
Searching for "All Shook Up" by Elvis Presley...
Done

No results found for: 'Wear My Ring Around Your Neck / Doncha' Think It's Time Elvis Presley'
Searching for "Poor Little Fool" by Ricky Nelson...
Done.
Searching for "Wonderful Time Up There / It's Too Soon To Know" by Pat Boone...
Done.
Searching for "Just A Dream" by Jimmy Clanton...
Done.
Searching for "Sugartime" by McGuire Sisters...
Done.
Searching for "Rockin Robbin / Over And Over" by Bobby Day...
Done.
Searching for "Tom Dooley" by Kingston Trio...
Done.
Searching for "Sweet Little Sixteen" by Chuck Berry...
Done.
Searching for "Topsy II / Topsy I" by Cozy Cole...
Done.
Searching for "Looking Back / Do I Like It" by Nat King Cole...
Done.
Searching for "Book Of Love" by Monotones...
Done.
Searching for "Tea For Two Cha Cha" by Tommy Dorsey Orch....
Done.
Searching for "Tears On My Pillow" by Little Anthony...
Done.
Searching for "Short Shorts" by Royal Teens...
Done.
Searching for "Great Balls Of Fire" by Jerry Lee Lewis...
Done.
Searching for "Lollipop" by Chordettes...
Done.

Searching for "Forty Miles Of Bad Road" by Duane Eddy...
Specified song does not contain lyrics. Rejecting.
Searching for "Just Ask Your Heart" by Frankie Avalon...
Done.
Searching for "Tell Him No" by Travis...
Done.
Searching for "Frankie" by Connie Francis...
Done.
Searching for "I've Had It" by Bell Notes...
Done.
Searching for "I Cried A Tear" by Lavern Baker...
Done.
Searching for "Enchanted" by Platters...
Done.
Searching for "Since I Don't Have You" by Skyliners...
Done.
Searching for "Peter Gunn Theme" by Ray Anthony...
Done.
Searching for "The Chipmunk Song" by David Seville...
Done.
Searching for "I Want To Walk You Home" by Fats Domino...
Done.
Searching for "So Fine" by Fiestas...
Done.
Searching for "Bobby Sox To Stockings" by Frankie Avalon...
Done.
Searching for "Deck Of Cards" by Wink Martindale...
Done.
Searching for "A Lover's Question" by Clyde Mcphatter...
Done.
Searching for "I Only Have Eyes For You" by Flamingos...
Done.
Searching for "It's Late" by Ricky Nelson

...

Done.
Searching for "Story Of My Life" by One Direction...
Done.
Searching for "Chandelier" by Sia...
Done.
Searching for "Black Widow" by Iggy Azalea feat. Rita Ora...
Done.
Searching for "Bang Bang" by Jessie J, Ariana Gr...
Done.
Searching for "Latch" by Disclosure feat. Sam Smith...
Done.
Searching for "Maps" by Maroon 5...
Done.
Searching for "Loyal" by Chris Brown feat. Lil Wayne...
Done.
Searching for "Best Day Of My Life" by American Authors...
Done.
Searching for "Habits (Stay High)" by Tove Lo...
Done.
Searching for "Summer" by Calvin Harris...
Done.
Searching for "Boom Clap" by Charli XCX...
Done.
Searching for "Drunk In Love" by Beyonce feat. Jay Z...
Done.
Searching for "Anaconda" by Nicki Minaj...
Done.
Searching for "Break Free" by Ariana Gr...
Done.
Searching for "Bailando" by Enrique Iglesias feat. Descemer Bueno...
Done.
Searching for "Burn" by Ellie Goulding...
Done.
Searching for "Wiggle" by Jason Derulo feat. Snoop Dogg...
Done.
Searching for "Rather Be" by Clean B

Done.
Searching for "Chains" by Nick Jonas...
Done.
Searching for "All Eyes On You" by Meek Mill feat. Chris Brown...
Done.
Searching for "No Type" by Rae Sremmurd...
Done.
Searching for "Riptide" by Vance Joy...
Done.
Searching for "Classic Man" by Jidenna feat. Roman GianArthur...
Done.
Searching for "Ex's and Oh's" by Elle King...
Done.
Searching for "Dear Future Husband" by Meghan Trainor...
Done.
Searching for "Marvin Gaye" by Charlie Puth feat. Meghan Trainor...
Done.
Searching for "Like I'm Gonna Lose You" by Meghan Trainor feat. John Legend...
Done.
Searching for "Habits (Stay High)" by Tove Lo...
Done.
Searching for "The Hanging Tree" by James Newton Howard feat. Jennifer Lawrence...
Done.
Searching for "Coco" by O.T. Genasis...
Done.
Searching for "Bang Bang" by Jessie J, Ariana Gr...
Done.
Searching for "Lay Me Down" by Sam Smith...
Done.
Searching for "Tuesday" by iLoveMakonnen feat. Drake...
Done.
Searching for "Hit The Quan" by iLoveMemphis...
Done.
Searching for "Downtow

Done.
Searching for "Location" by Khalid...
Done.
Searching for "Attention" by Charlie Puth...
Done.
Searching for "There's Nothing Holdin' Me Back" by Shawn Mendes...
Done.
Searching for "Bodak Yellow (Money Moves)" by Cardi B...
Done.
Searching for "Redbone" by Childish Gambino...
Done.
Searching for "I Don't Wanna Live Forever (Fifty Shades Darker)" by Zayn / Taylor Swift...
Done.
Searching for "It Ain't Me" by Kygo x Selena Gomez...
Done.
Searching for "iSpy" by KYLE...
Done.
Searching for "Issues" by Julia Michaels...
Done.
Searching for "Scars To Your Beautiful" by Alessia Cara...
Done.
Searching for "1-800-273-8255" by Logic...
Done.
Searching for "Slow Hands" by Niall Horan...
Done.
Searching for "Love On The Brain" by Rihanna...
Done.
Searching for "I Feel It Coming" by The Weeknd...
Done.
Searching for "Bounce Back" by Big Sean...
Done.
Searching for "Strip That Down" by Liam Payne...
Done.
Searching for "Fake Love" by Drake...
Done.
Searching for "Don't Wanna Know" by Maroon

Done.
Searching for "Eastside" by benny blanco, Halsey & Khalid...
Done.
Searching for "Call Out My Name" by The Weeknd...
Done.
Searching for "King's Dead" by Jay Rock, Kendrick Lamar, Future & James Blake...
Done.
Searching for "Happier" by Marshmello & Bastille...
Done.
Searching for "Te Bote" by Casper Magico, Nio Garcia, Darell, Nicky Jam, Ozuna & Bad Bunny...
Done.
Searching for "Simple" by Florida Georgia Line...
Done.
Searching for "Lemon" by N*E*R*D & Rihanna...
Done.
Searching for "1-800-273-8255" by Logic...
Done.
Searching for "Say Something" by Justin Timberlake...
Done.
Searching for "I'm Upset" by Drake...
Done.
Searching for "Get Along" by Kenny Chesney...
Done.
Searching for "Moonlight" by XXXTENTACION...
Done.
Searching for "What Lovers Do" by Maroon 5...
Done.
Searching for "X" by Nicky Jam x J Balvin...
Done.
Searching for "Outside Today" by YoungBoy Never Broke Again...
Done.
Searching for "Trip" by Ella Mai...
Done.
Searching for "Dura" by Daddy Yankee...
Done.
Se

Done.
Searching for "Heartless" by The Weeknd...
Done.
Searching for "BOP" by DaBaby...
Done.
Searching for "Lose You To Love Me" by Selena Gomez...
Done.
Searching for "Good As Hell" by Lizzo...
Done.
Searching for "Toosie Slide" by Drake...
Done.
Searching for "Break My Heart" by Dua Lipa...
Done.
Searching for "Chasin' You" by Morgan Wallen...
Done.
Searching for "Savage Love (Laxed - Siren Beat)" by Jawsh 685 x Jason Derulo...
Done.
Searching for "No Guidance" by Chris Brown...
Done.
Searching for "My Oh My" by Camila Cabello...
Done.
Searching for "Dynamite" by BTS...
Done.
Searching for "Go Crazy" by Chris Brown & Young Thug...
Done.
Searching for "High Fashion" by Roddy Ricch...
Done.
Searching for "Laugh Now Cry Later" by Drake...
Done.
Searching for "Woah" by Lil Baby...
Done.
Searching for "Death Bed" by Powfu...
Done.
Searching for "Senorita" by Shawn Mendes & Camila Cabello...
Done.
Searching for "HIGHEST IN THE ROOM" by Travis Scott...
Done.
Searching for "Bad Guy" by Bill

In [27]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
# err based ong error_list
# Fixed 6140 Beyonce 7/11 (date format in csv)
err = [12, 23, 25, 53, 60, 74, 84, 100, 101, 103, 111, 116, 141, 160, 173, 190, 196, 210, 233, 239, 266, 277, 334, 365, 380, 385, 386, 401, 428, 439, 440, 448, 501, 515, 519, 537, 554, 565, 604, 616, 619, 633, 690, 694, 696, 708, 723, 771, 774, 784, 800, 816, 832, 838, 853, 862, 866, 871, 906, 920, 937, 969, 991, 1039, 1111, 1126, 1150, 1180, 1211, 1245, 1277, 1381, 1387, 1397, 1400, 1422, 1454, 1537, 1542, 1569, 1598, 1613, 1615, 1668, 1801, 1807, 1822, 1850, 1873, 1895, 1940, 1965, 1982, 2027, 2095, 2144, 2167, 2177, 2189, 2208, 2269, 2350, 2370, 2400, 2528, 2559, 2603, 2633, 2690, 2738, 2791, 2835, 2843, 3140, 3334, 3460, 3742, 3908, 3952, 4121, 4169, 4171, 4180, 4208, 4244, 4245, 4280, 4351, 4361, 4387, 4518, 4579, 4760, 4777, 4823, 5134, 5235]
display(all_time_billboard_wrap_up.iloc[err])

,Position,Artist,Song Title,Year,Split names
12,13,Gordon Jenkins and The Weavers,"Tzena, Tzena, Tzena",1950,Gordon Jenkins
23,24,Bill Snyder,Bewitched,1950,Bill Snyder
25,26,Gordon Jenkins,Bewitched,1950,Gordon Jenkins
53,24,Del Wood,Down Yonder,1951,Del Wood
60,1,Leroy Anderson,Blue Tango,1952,Leroy Anderson
74,15,Jo Stafford,Jambalaya,1952,Jo Stafford
84,25,Les Paul,Meet Mr. Callaghan,1952,Les Paul
100,11,Pee Wee Hunt,Oh,1953,Pee Wee Hunt
101,12,Frank Chacksfield,Ebb Tide,1953,Frank Chacksfield
103,14,Richard Hayman,Ruby,1953,Richard Hayman
